In [1]:
pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 92.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import re
import requests
import pandas as pd
from PIL import Image
from transformers import AutoTokenizer, AutoModel, CLIPImageProcessor
import torch

# Load the model and tokenizer
path = "OpenGVLab/InternVL2-8B"
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True).eval().cuda()
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)
image_processor = CLIPImageProcessor.from_pretrained(path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

configuration_internvl_chat.py:   0%|          | 0.00/3.85k [00:00<?, ?B/s]

configuration_internlm2.py:   0%|          | 0.00/7.00k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL2-8B:
- configuration_internlm2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL2-8B:
- tokenization_internlm2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer.model:   0%|          | 0.00/1.48M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

In [4]:
import pandas as pd
import requests
from PIL import Image
import torch
import os
import re

# Define entity unit map
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon',
                    'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

# Function to standardize unit
def standardize_unit(unit):
    unit_map = {
        'cm': 'centimetre', 'm': 'metre', 'mm': 'millimetre', 'ft': 'foot', 'in': 'inch', 'yd': 'yard', 'inches': 'inch','"':'inch',
        'g': 'gram', 'kg': 'kilogram', 'mg': 'milligram', 'mcg': 'microgram', 'oz': 'ounce', 'lb': 'pound', 'tonne': 'ton',
        'v': 'volt', 'kv': 'kilovolt', 'mv': 'millivolt', 'w': 'watt', 'kw': 'kilowatt', 'l': 'litre', 'ml': 'millilitre',
        'cl': 'centilitre', 'dl': 'decilitre', 'fl oz': 'fluid ounce', 'gal': 'gallon', 'imp gal': 'imperial gallon',
        'cu in': 'cubic inch', 'cu ft': 'cubic foot', 'pt': 'pint', 'qt': 'quart'
    }
    return unit_map.get(unit.lower(), unit)

# Function to extract value and unit from text
def extract_value_and_unit(text, entity_name):
    # Define regex pattern for extracting value and unit
    pattern = r'(\d+\.?\d*)\s*([a-zA-Z]+)'
    matches = re.findall(pattern, text)

    for match in matches:
        value, unit = match
        unit = standardize_unit(unit)
        if unit in entity_unit_map[entity_name]:
            return value, unit
    return None, None

# Function to process the CSV and prepare for download
def process_csv(input_csv_path, output_csv_path):
    df = pd.read_csv(input_csv_path)
    entity_values = []

    for index, row in df.iterrows():
        image_url = row['image_link']
        entity_name = row['entity_name']

        # Download the image
        image_path = f"{index}.jpg"
        response = requests.get(image_url)
        with open(image_path, 'wb') as f:
            f.write(response.content)

        # Open and preprocess the image
        image = Image.open(image_path).resize((448, 448))
        pixel_values = image_processor(images=image, return_tensors='pt').pixel_values.to(torch.bfloat16).cuda()

        # Ask the LLM the question about the entity
        questions = [f'<image>\n answer in one line :Analyse the product in this image, and write the most appropriate width, only return dimension if the object has a width else just say none']
        responses = model.batch_chat(tokenizer, pixel_values, num_patches_list=[pixel_values.size(0)],
                                     questions=questions, generation_config={'max_new_tokens': 1024, 'do_sample': False})

        # Extract value and unit
        response_text = responses[0]
        print(response_text)
        value, unit = extract_value_and_unit(response_text, entity_name)

        # Append result to list
        if value and unit:
            entity_values.append(f"{value} {unit}")
        else:
            entity_values.append(None)

        # Print link and output for verification
        print(f"Image URL: {image_url} | Extracted Output: {entity_values[-1]}")

        # Clean up the downloaded image
        os.remove(image_path)

    # Add the entity values to a new column
    df['entity_value'] = entity_values

    # Save processed CSV to the specified output file path
    df.to_csv(output_csv_path, index=False)

    return output_csv_path



In [5]:
processed_csv_path = process_csv('/content/drive/MyDrive/amzn-data/width/width_batch_4.csv', '/content/width_4.csv')




Streaming output truncated to the last 5000 lines.
23.6 inches
Image URL: https://m.media-amazon.com/images/I/51xrZK2lUDL.jpg | Extracted Output: 23.6 inch
The most appropriate width of the product in the image is 30.7 inches.
Image URL: https://m.media-amazon.com/images/I/51xrdCIWTFL.jpg | Extracted Output: 30.7 inch
12"
Image URL: https://m.media-amazon.com/images/I/51xrmKjSbXL.jpg | Extracted Output: None
12cm
Image URL: https://m.media-amazon.com/images/I/51xspCVJZYL.jpg | Extracted Output: 12 centimetre
The most appropriate width of the product in the image is 14 cm / 5.5 inches.
Image URL: https://m.media-amazon.com/images/I/51xtJxcV-1L.jpg | Extracted Output: 14 centimetre
None
Image URL: https://m.media-amazon.com/images/I/51xtQb4ONTL.jpg | Extracted Output: None
1-1/6"
Image URL: https://m.media-amazon.com/images/I/51xtW-4OD4L.jpg | Extracted Output: None
None
Image URL: https://m.media-amazon.com/images/I/51xtilknfhL.jpg | Extracted Output: None
4.2"
Image URL: https://m.medi

In [6]:
from google.colab import files

# Provide a link to download the processed CSV in Colab
files.download(processed_csv_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
processed_csv_path = process_csv('/content/drive/MyDrive/amzn-data/width/width_batch_5.csv', '/content/width_5.csv')




Streaming output truncated to the last 5000 lines.
55mm
Image URL: https://m.media-amazon.com/images/I/617DR-NbdWL.jpg | Extracted Output: 55 millimetre
The most appropriate width of the product in the image is 8.66" (22 cm).
Image URL: https://m.media-amazon.com/images/I/617DVRfRj8L.jpg | Extracted Output: 22 centimetre
The most appropriate width for the product in this image is 196mm.
Image URL: https://m.media-amazon.com/images/I/617DWc9DaHL.jpg | Extracted Output: 196 millimetre
35cm/13.7 inch
Image URL: https://m.media-amazon.com/images/I/617DmhnLUsL.jpg | Extracted Output: 35 centimetre
None
Image URL: https://m.media-amazon.com/images/I/617DsI9669L.jpg | Extracted Output: None
None
Image URL: https://m.media-amazon.com/images/I/617E-cxDANL.jpg | Extracted Output: None
50cm
Image URL: https://m.media-amazon.com/images/I/617EDdIwacL.jpg | Extracted Output: 50 centimetre
The most appropriate width of the product in the image is 12cm.
Image URL: https://m.media-amazon.com/images/I/6

In [8]:
from google.colab import files

# Provide a link to download the processed CSV in Colab
files.download(processed_csv_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
processed_csv_path = process_csv('/content/drive/MyDrive/amzn-data/width/width_batch_6.csv', '/content/width_6.csv')

# Provide a link to download the processed CSV in Colab
files.download(processed_csv_path)


The most appropriate width for the product in the image is 6 inches.
Image URL: https://m.media-amazon.com/images/I/61CIVwoNJvL.jpg | Extracted Output: 6 inch
10mm
Image URL: https://m.media-amazon.com/images/I/61CIwQqhY9L.jpg | Extracted Output: 10 millimetre
The most appropriate width of the product in the image is 43 inches.
Image URL: https://m.media-amazon.com/images/I/61CJJ60Ap1L.jpg | Extracted Output: 43 inch
None
Image URL: https://m.media-amazon.com/images/I/61CJh8OiOdL.jpg | Extracted Output: None
1.25ft (38cm)
Image URL: https://m.media-amazon.com/images/I/61CK8AgYDoL.jpg | Extracted Output: 1.25 foot
None
Image URL: https://m.media-amazon.com/images/I/61CKElATKOL.jpg | Extracted Output: None
16.4cm
Image URL: https://m.media-amazon.com/images/I/61CKJt5YlDL.jpg | Extracted Output: 16.4 centimetre
The most appropriate width for the product in this image is 25cm.
Image URL: https://m.media-amazon.com/images/I/61CKTEtlKWL.jpg | Extracted Output: 25 centimetre
The most appropri

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
import pandas as pd
import requests
from PIL import Image
import torch
import os
import re

# Define entity unit map
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon',
                    'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

# Function to standardize unit
def standardize_unit(unit):
    unit_map = {
        'cm': 'centimetre', 'm': 'metre', 'mm': 'millimetre', 'ft': 'foot', 'in': 'inch', 'yd': 'yard', 'inches': 'inch','"':'inch',
        'g': 'gram', 'kg': 'kilogram', 'mg': 'milligram', 'mcg': 'microgram', 'oz': 'ounce', 'lb': 'pound', 'tonne': 'ton',
        'v': 'volt', 'kv': 'kilovolt', 'mv': 'millivolt', 'w': 'watt', 'kw': 'kilowatt', 'l': 'litre', 'ml': 'millilitre',
        'cl': 'centilitre', 'dl': 'decilitre', 'fl oz': 'fluid ounce', 'gal': 'gallon', 'imp gal': 'imperial gallon',
        'cu in': 'cubic inch', 'cu ft': 'cubic foot', 'pt': 'pint', 'qt': 'quart'
    }
    return unit_map.get(unit.lower(), unit)

# Function to extract value and unit from text
def extract_value_and_unit(text, entity_name):
    # Define regex pattern for extracting value and unit
    pattern = r'(\d+\.?\d*)\s*([a-zA-Z]+)'
    matches = re.findall(pattern, text)

    for match in matches:
        value, unit = match
        unit = standardize_unit(unit)
        if unit in entity_unit_map[entity_name]:
            return value, unit
    return None, None

# Function to process the CSV and prepare for download
def process_csv(input_csv_path, output_csv_path):
    df = pd.read_csv(input_csv_path)
    entity_values = []

    for index, row in df.iterrows():
        image_url = row['image_link']
        entity_name = row['entity_name']

        # Download the image
        image_path = f"{index}.jpg"
        response = requests.get(image_url)
        with open(image_path, 'wb') as f:
            f.write(response.content)

        # Open and preprocess the image
        image = Image.open(image_path).resize((448, 448))
        pixel_values = image_processor(images=image, return_tensors='pt').pixel_values.to(torch.bfloat16).cuda()

        # Ask the LLM the question about the entity
        questions = [f'<image>\n answer in one line :Analyse the product in this image, and write the most appropriate height of the object present otherwise say none']
        responses = model.batch_chat(tokenizer, pixel_values, num_patches_list=[pixel_values.size(0)],
                                     questions=questions, generation_config={'max_new_tokens': 1024, 'do_sample': False})

        # Extract value and unit
        response_text = responses[0]
        print(response_text)
        value, unit = extract_value_and_unit(response_text, entity_name)

        # Append result to list
        if value and unit:
            entity_values.append(f"{value} {unit}")
        else:
            entity_values.append(None)

        # Print link and output for verification
        print(f"Image URL: {image_url} | Extracted Output: {entity_values[-1]}")

        # Clean up the downloaded image
        os.remove(image_path)

    # Add the entity values to a new column
    df['entity_value'] = entity_values

    # Save processed CSV to the specified output file path
    df.to_csv(output_csv_path, index=False)

    return output_csv_path

In [11]:
processed_csv_path = process_csv('/content/drive/MyDrive/amzn-data/height/height_batch_5.csv', '/content/height_5.csv')

from google.colab import files
files.download(processed_csv_path)


Streaming output truncated to the last 5000 lines.
The height of the object in the image is 3ft (90cm).
Image URL: https://m.media-amazon.com/images/I/61-sKeai23L.jpg | Extracted Output: 3 foot
The most appropriate height of the objects present in the image is 3.5 inches.
Image URL: https://m.media-amazon.com/images/I/61-sTcqjzoL.jpg | Extracted Output: 3.5 inch
The most appropriate height of the object present in the image is 30 cm.
Image URL: https://m.media-amazon.com/images/I/61-sWJiDiXL.jpg | Extracted Output: 30 centimetre
The most appropriate height of the object present in the image is 600mm (23.6 inches).
Image URL: https://m.media-amazon.com/images/I/61-sZaSdX1L.jpg | Extracted Output: 600 millimetre
The most appropriate height of the object present in the image is 13.2 cm.
Image URL: https://m.media-amazon.com/images/I/61-sce6eEHL.jpg | Extracted Output: 13.2 centimetre
The most appropriate height of the object present in the image is 12 cm (4.7 inches).
Image URL: https://m

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
processed_csv_path = process_csv('/content/drive/MyDrive/amzn-data/height/height_batch_6.csv', '/content/height_6.csv')

from google.colab import files
files.download(processed_csv_path)


Streaming output truncated to the last 5000 lines.
The most appropriate height of the object present in the image is 2.5 cm.
Image URL: https://m.media-amazon.com/images/I/618AwN4UzML.jpg | Extracted Output: 2.5 centimetre
The most appropriate height of the objects present in the image is 45 cm.
Image URL: https://m.media-amazon.com/images/I/618B2jwFsuS.jpg | Extracted Output: 45 centimetre
The most appropriate height of the object present in the image is approximately 40x60 cm.
Image URL: https://m.media-amazon.com/images/I/618B2lLKTHL.jpg | Extracted Output: 60 centimetre
The most appropriate height of the object present in the image is 4.5 feet.
Image URL: https://m.media-amazon.com/images/I/618BBmOXDGL.jpg | Extracted Output: None
The height of the object in the image is 14 cm.
Image URL: https://m.media-amazon.com/images/I/618BDaKOIHL.jpg | Extracted Output: 14 centimetre
The most appropriate height of the object present in the image is 2.5 cm or 0.98 inches.
Image URL: https://m.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
import pandas as pd
import requests
from PIL import Image
import torch
import os
import re

# Define entity unit map
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon',
                    'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

# Function to standardize unit
def standardize_unit(unit):
    unit_map = {
        'cm': 'centimetre', 'm': 'metre', 'mm': 'millimetre', 'ft': 'foot', 'in': 'inch', 'yd': 'yard', 'inches': 'inch','"':'inch',
        'g': 'gram', 'kg': 'kilogram', 'mg': 'milligram', 'mcg': 'microgram', 'oz': 'ounce', 'lb': 'pound', 'tonne': 'ton',
        'v': 'volt', 'kv': 'kilovolt', 'mv': 'millivolt', 'w': 'watt', 'kw': 'kilowatt', 'l': 'litre', 'ml': 'millilitre',
        'cl': 'centilitre', 'dl': 'decilitre', 'fl oz': 'fluid ounce', 'gal': 'gallon', 'imp gal': 'imperial gallon',
        'cu in': 'cubic inch', 'cu ft': 'cubic foot', 'pt': 'pint', 'qt': 'quart', "volts":"volt", "vac": "volt", "watts": "watt"
    }
    return unit_map.get(unit.lower(), unit)

# Function to extract value and unit from text
def extract_value_and_unit(text, entity_name):
    # Define regex pattern for extracting value and unit
    pattern = r'(\d+\.?\d*)\s*([a-zA-Z]+)'
    matches = re.findall(pattern, text)

    for match in matches:
        value, unit = match
        unit = standardize_unit(unit)
        if unit in entity_unit_map[entity_name]:
            return value, unit
    return None, None

# Function to process the CSV and prepare for download
def process_csv(input_csv_path, output_csv_path):
    df = pd.read_csv(input_csv_path)
    entity_values = []

    for index, row in df.iterrows():
        image_url = row['image_link']
        entity_name = row['entity_name']

        # Download the image
        image_path = f"{index}.jpg"
        response = requests.get(image_url)
        with open(image_path, 'wb') as f:
            f.write(response.content)

        # Open and preprocess the image
        image = Image.open(image_path).resize((448, 448))
        pixel_values = image_processor(images=image, return_tensors='pt').pixel_values.to(torch.bfloat16).cuda()

        # Ask the LLM the question about the entity
        questions = [f'<image>\n answer in one line :find the wattage of the product in the image if found otherwise say none']
        responses = model.batch_chat(tokenizer, pixel_values, num_patches_list=[pixel_values.size(0)],
                                     questions=questions, generation_config={'max_new_tokens': 1024, 'do_sample': False})

        # Extract value and unit
        response_text = responses[0]
        print(response_text)
        value, unit = extract_value_and_unit(response_text, entity_name)

        # Append result to list
        if value and unit:
            entity_values.append(f"{value} {unit}")
        else:
            entity_values.append(None)

        # Print link and output for verification
        print(f"Image URL: {image_url} | Extracted Output: {entity_values[-1]}")

        # Clean up the downloaded image
        os.remove(image_path)

    # Add the entity values to a new column
    df['entity_value'] = entity_values

    # Save processed CSV to the specified output file path
    df.to_csv(output_csv_path, index=False)

    return output_csv_path



In [16]:
processed_csv_path = process_csv('/content/wattage.csv', '/content/wattage_mod.csv')

from google.colab import files
files.download(processed_csv_path)


Streaming output truncated to the last 5000 lines.
None
Image URL: https://m.media-amazon.com/images/I/51wvpDQSwrL.jpg | Extracted Output: None
600W
Image URL: https://m.media-amazon.com/images/I/51wx6cYev7L.jpg | Extracted Output: 600 watt
25W
Image URL: https://m.media-amazon.com/images/I/51x14O6zinL.jpg | Extracted Output: 25 watt
None
Image URL: https://m.media-amazon.com/images/I/51x1ebNOUcL.jpg | Extracted Output: None
The wattage of the product in the image is 0.24 W.
Image URL: https://m.media-amazon.com/images/I/51x2sLqwJMS.jpg | Extracted Output: 0.24 watt
1400W
Image URL: https://m.media-amazon.com/images/I/51x5holN9OL.jpg | Extracted Output: 1400 watt
20 Watts
Image URL: https://m.media-amazon.com/images/I/51x7-XtDyNL.jpg | Extracted Output: 20 watt
The wattage of the product in the image is 45 Watts.
Image URL: https://m.media-amazon.com/images/I/51x8drs5yuL.jpg | Extracted Output: 45 watt
The wattage of the product in the image is 530W.
Image URL: https://m.media-amazon.c

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [41]:
import pandas as pd
import requests
from PIL import Image
import torch
import os
import re

# Define entity unit map
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon',
                    'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

# Function to standardize unit
def standardize_unit(unit):
    unit_map = {
        'cm': 'centimetre', 'm': 'metre', 'mm': 'millimetre', 'ft': 'foot', 'in': 'inch', 'yd': 'yard', 'inches': 'inch','"':'inch',
        'g': 'gram', 'kg': 'kilogram', 'mg': 'milligram', 'mcg': 'microgram', 'oz': 'ounce', 'lb': 'pound', 'tonne': 'ton',
        'v': 'volt', 'kv': 'kilovolt', 'mv': 'millivolt', 'w': 'watt', 'kw': 'kilowatt', 'l': 'litre', 'ml': 'millilitre',
        'cl': 'centilitre', 'dl': 'decilitre', 'fl oz': 'fluid ounce', 'gal': 'gallon', 'imp gal': 'imperial gallon',
        'cu in': 'cubic inch', 'cu ft': 'cubic foot', 'pt': 'pint', 'qt': 'quart'
    }
    return unit_map.get(unit.lower(), unit)

# Function to extract value and unit from text
def extract_value_and_unit(text, entity_name):
    # Define regex pattern for extracting value and unit
    pattern = r'(\d+\.?\d*)\s*([a-zA-Z]+)'
    matches = re.findall(pattern, text)

    for match in matches:
        value, unit = match
        unit = standardize_unit(unit)
        if unit in entity_unit_map[entity_name]:
            return value, unit
    return None, None

# Function to process the CSV and prepare for download
def process_csv(input_csv_path, output_csv_path):
    df = pd.read_csv(input_csv_path)
    entity_values = []

    for index, row in df.iterrows():
        image_url = row['image_link']
        entity_name = row['entity_name']

        # Download the image
        image_path = f"{index}.jpg"
        response = requests.get(image_url)
        with open(image_path, 'wb') as f:
            f.write(response.content)

        # Open and preprocess the image
        image = Image.open(image_path).resize((448, 448))
        pixel_values = image_processor(images=image, return_tensors='pt').pixel_values.to(torch.bfloat16).cuda()

        # Ask the LLM the question about the entity
        questions = [f'<image>\n answer in one line : does the image looks like a container? if so what is the height mentioned(mention height only if container)? if not a container, just say no']
        responses = model.batch_chat(tokenizer, pixel_values, num_patches_list=[pixel_values.size(0)],
                                     questions=questions, generation_config={'max_new_tokens': 1024, 'do_sample': False})

        # Extract value and unit
        response_text = responses[0]
        print(response_text)
        value, unit = extract_value_and_unit(response_text, entity_name)

        # Append result to list
        if value and unit:
            entity_values.append(f"{value} {unit}")
        else:
            entity_values.append(None)

        # Print link and output for verification
        print(f"Image URL: {image_url} | Extracted Output: {entity_values[-1]}")

        # Clean up the downloaded image
        os.remove(image_path)

    # Add the entity values to a new column
    df['entity_value'] = entity_values

    # Save processed CSV to the specified output file path
    df.to_csv(output_csv_path, index=False)

    return output_csv_path

In [39]:
# prompt: split /content/drive/MyDrive/amzn-data/depth/depth_batch_1.csv into multiple batches of 1005

import pandas as pd

def split_csv(input_file, batch_size):
  """
  Splits a CSV file into multiple batches.

  Args:
    input_file: Path to the input CSV file.
    batch_size: Size of each batch.
  """
  df = pd.read_csv(input_file)
  num_batches = len(df) // batch_size + (len(df) % batch_size > 0)
  for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(df))
    batch_df = df[start_idx:end_idx]
    output_file = f"/content/depth_batch_2_split_{i+1}.csv"
    batch_df.to_csv(output_file, index=False)
    print(f"Saved batch {i+1} to {output_file}")

# Split the CSV file into batches of 1005 rows
split_csv('/content/drive/MyDrive/amzn-data/depth/depth_batch_2.csv', 1005)


Saved batch 1 to /content/depth_batch_2_split_1.csv
Saved batch 2 to /content/depth_batch_2_split_2.csv
Saved batch 3 to /content/depth_batch_2_split_3.csv
Saved batch 4 to /content/depth_batch_2_split_4.csv
Saved batch 5 to /content/depth_batch_2_split_5.csv


In [ ]:
# prompt: run this function on all the batches above processed_csv_path = process_csv('/content/drive/MyDrive/amzn-data/depth/depth_batch_1.csv', '/content/depth_1.csv')
# from google.colab import files
# files.download(processed_csv_path)

# Assuming the split files are named depth_batch_1_split_*.csv
import glob

# Get a list of all split CSV files
split_files = glob.glob('/content/depth_batch_2_split_*.csv')

# Process each split file
for file in split_files:
  # Extract the batch number from the filename
  batch_number = file.split('_')[-1].split('.')[0]
  output_file = f'/content/depth_{batch_number}.csv'

  # Process the CSV file
  processed_csv_path = process_csv(file, output_file)

  # Download the processed CSV
  files.download(processed_csv_path)


No, the image does not look like a container.
Image URL: https://m.media-amazon.com/images/I/51BYrWsc7jL.jpg | Extracted Output: None
Yes, the image looks like a container. The height mentioned is 7.8 cm.
Image URL: https://m.media-amazon.com/images/I/51BYvwdAetL.jpg | Extracted Output: 7.8 centimetre
Yes, the image looks like a container. The height mentioned is 26cm/10.23in.
Image URL: https://m.media-amazon.com/images/I/51BZHXZ2NCL.jpg | Extracted Output: 26 centimetre
No, the image does not look like a container.
Image URL: https://m.media-amazon.com/images/I/51BZPUjovmL.jpg | Extracted Output: None
Yes, the image looks like a container. The height mentioned is 9 cm.
Image URL: https://m.media-amazon.com/images/I/51BZi3cW1US.jpg | Extracted Output: 9 centimetre
No, the image does not look like a container.
Image URL: https://m.media-amazon.com/images/I/51BZkRdpqcL.jpg | Extracted Output: None
Yes, the image looks like a container. The height mentioned is 5 inches.
Image URL: https: